# Colab: Роутер по типу запроса

**Цель:** автоматически выбирать адаптер по ключевым словам в сообщении пользователя.

## 0. Установка и загрузка модели

In [ ]:
!pip install -q transformers peft torch

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

BASE_MODEL = "mistralai/Mistral-7B-v0.1"
ADAPTER_PATHS = {
    "json": "outputs_finetome/final",     # подставьте свой путь
    "coding": "outputs_finetome/final",   # или разные адаптеры
    "support": "outputs_finetome/final",
}

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", torch_dtype=torch.float16)
model = PeftModel.from_pretrained(base_model, ADAPTER_PATHS["json"], adapter_name="json")

for name, path in ADAPTER_PATHS.items():
    if name != "json":
        try:
            model.load_adapter(path, adapter_name=name)
        except Exception:
            pass

print("Адаптеры:", list(model.peft_config.keys()))

## 1. Роутер

In [ ]:
def get_adapter_for_prompt(text: str) -> str:
    """Выбор адаптера по ключевым словам."""
    t = text.lower()
    if any(k in t for k in ["json", "схем", "структур", "api", "извлеч", "парсинг"]):
        return "json"
    if any(k in t for k in ["код", "функци", "bug", "рефакторинг", "напиши", "python"]):
        return "coding"
    if any(k in t for k in ["помощь", "ошибк", "не работа", "поддержка", "вход"]):
        return "support"
    return "json"  # default


def chat(user_message: str, max_tokens: int = 120) -> str:
    adapter = get_adapter_for_prompt(user_message)
    model.set_adapter(adapter)
    print(f"[Адаптер: {adapter}]")
    
    prompt = f"### Instruction:\n{user_message}\n\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(**inputs, max_new_tokens=max_tokens, do_sample=True, temperature=0.7, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(out[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

In [ ]:
# Примеры (если все адаптеры одинаковые — роутер всё равно переключает)
queries = [
    "Извлеки в JSON: имя, возраст, город",
    "Напиши функцию для сортировки списка",
    "Пользователь не может войти в аккаунт, что делать?",
]

for q in queries:
    print(f"\nВопрос: {q}")
    print(chat(q))
    print("---")